
# <center>  Proyecto Final - Fase 4  </center>

## Descripción
- En esta sección construiremos 3 redes neuronales profundas para un problema de clasificación.
- Trabajeremos con las librerías Tensorflow y Keras.
  


## Problema de negocio

    
Un problema imperante en el entorno bancario a nivel global se enfoca en el riesgo crediticio o riesgo de impago. 
  
Este aspecto hace refencia a las pérdidas que incurren los bancos dado que los prestamistas son incapaces de devolver los créditos otorgados. 

Lo cual conlleva la siguiente pregunta: **¿Cómo poder reducir el número de personas que caen en mora crediticia?** 
  
Aspecto que se aborda a continuación.

## Objetivos del proyecto


El objetivo de este proyecto es construir un modelo que los prestatarios puedan usar para ayudar a tomar las mejores decisiones financieras.

Mediante el uso de diferentes algoritmos se obtendrá la probabilidad de impago en cada individuo. El principal indicador que se va usar para ver la bonanza de los distintos modelos va a ser el AUC (Area Under the Curve) generado por las curvas ROC.

## Importar librerías y verificar versiones

In [0]:
import sys
import numpy as np
import pandas as pd
import sklearn as sk
import tensorflow as tf
from tensorflow import keras
# Librerías para visualización
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

In [0]:
print('** Versiones Actuales                     | Requeridas **')
print('Python:\nVersion Actual:', sys.version, ' | Requerida >= 3.6')
print('NumPy:\nVersion Actual: {:10}             | Requerida >= 1.16.2'.format(np.__version__))
print('Pandas:\nVersion Actual: {:10}            | Requerida >= 0.24.2'.format(pd.__version__))
print('Scikit-learn:\nVersion Actual: {:10}      | Requerida >= 1.2.1'.format(sk.__version__))
print('Matplotlib:\nVersion Actual: {:10}        | Requerida >= 3.0.3'.format(matplotlib.__version__))
print('Seaborn:\nVersion Actual: {:10}           |Requerida >= 0.9.0 '.format(sns.__version__))

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt

## Descripción de las variables contenidas en el _dataset_


El _dataset_ usado proviene de la plataforma **Kaggle**, y se encuentra separado en dos partes. 

La primera parte contiene los datos que serán usados para el entrenamiento del modelo, que serán nombrados como *datos_training*. La segunda parte son los datos que se usarán para el test del modelo *datos_test*. Estas muestras contienen 150.000 y 101.503 registros, respectivamente (aunque existen algunos campos que tienen distintos valores ausentes). 

Accesso a la Competencia **Give Me Some Credit:** [Link de acceso](https://www.kaggle.com/c/GiveMeSomeCredit/overview)

Link de descarga correspondiente a cada dataset: 
* [training data](https://storage.googleapis.com/datasets-academy/Track%20Data%20Science/Proyecto%20Final/cs-training.csv)
* [test data](https://storage.googleapis.com/datasets-academy/Track%20Data%20Science/Proyecto%20Final/cs-test.csv)

* **SeriousDlqin2yrs:** Persona que tiene 90 días de morosidad vencida o peor (Si = 1 y  No = 0)
* **RevolvingUtilizationOfUnsecuredLines:** El saldo total en tarjetas de crédito y líneas de crédito personales (excepto bienes inmuebles y sin deuda a plazos) sobre la suma de los límites de crédito
* **Age:** Edad prestamista en años 
* **NumberOfTime30-59DaysPastDueNotWorse:** Número de veces que el prestatario ha caido en mora entre 30 y 59 días, pero no ha empeorado en los últimos 2 años 
* **DebtRatio:** Pagos mensuales de deuda sobre el ingreso bruto mensual 
* **MonthlyIncome:** Ingreso mensual 
* **NumberOfOpenCreditLinesAndLoans:** Número de préstamos abiertos (cuotas como préstamos para automóviles o hipotecas) y líneas de crédito 
* **NumberOfTimes90DaysLate:** Número de veces que el prestatario ha caido en mora entre 90 días o más 
* **NumberRealEstateLoansOrLines:** Número de préstamos hipotecarios e inmobiliarios, incluidas las líneas de crédito hipotecario 
* **NumberOfTime60-89DaysPastDueNotWorse:** Número de veces que el prestatario ha caido en mora entre 60 y 89 días, pero no ha empeorado en los últimos 2 años 
* **NumberOfDependents:** Número de dependientes en la familia excluyendose a si mismos (ej: cónyuge, hijos, etc.)

## Carga de los datos

In [0]:
# Colocar la ruta del directorio correspondiente a cada dataset
ruta_training = 'https://storage.googleapis.com/datasets-academy/Track%20Data%20Science/Proyecto%20Final/cs-training.csv'
ruta_test     = 'https://storage.googleapis.com/datasets-academy/Track%20Data%20Science/Proyecto%20Final/cs-test.csv'

In [0]:
# Lectura de los datos usando pandas (creación de DataFrames)
datos_training = pd.read_csv(ruta_training, sep = ',', decimal = '.', index_col = 0)
datos_test     = pd.read_csv(ruta_test,     sep = ',', decimal = '.', index_col = 0)

In [0]:
# Imprimir el número de filas y columnas de ambos datasets
print('Filas y columnas - datos de entrenamiento', datos_training.shape)
print('Filas y columnas - datos de test', datos_test.shape)

In [0]:
# Imprimir los tipos de datos de cada columna en el dataset de entrenamiento
print(datos_training.dtypes)

## Ingeniería de variables

### Selección de variable objetivo y predictores del DataFrame `datos_training`

In [0]:
# En la variable y colocar la variable objetivo. En la variable X los predictores
y = datos_training.loc[:,'SeriousDlqin2yrs']
X = datos_training.loc[:, [i for i in datos_training.columns if i != 'SeriousDlqin2yrs']]

### División en datos de training y test

Para dividir la muestra de entrenamiento en dos submuestras de _training_ y _test_, se decide guardar una proporción de 80% _training_ 20% _test_. 

En los argumentos de `train_test_split` emplear el muestreo estratificado `stratify=y` y  `random_state=0`

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

In [0]:
# Imprimir las filas y columnas de los datos de training y test: para los predictores y para la variables objetivo
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### Escalamiento (variables numéricas)

In [0]:
# Selección de las etiquetas de las variables numéricas
X_num = X.select_dtypes(include=np.number).columns
print(X_num)

### Pipeline - Ingeniería de variable
A diferencia de los ejercicios anteriores, ejecutaremos directamente el segmento de ingeniería de variables en los datos de entrenamiento y test.

In [0]:
# Creación de un Pipeline para el preprocesamiento de las variables numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',  MinMaxScaler(feature_range=(0, 1)))])

In [0]:
# Delimitar el preprocesamiento de variables
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, X_num)])

In [0]:
#Transformación 
t = numeric_transformer.fit(X_train[X_num])

X_train = numeric_transformer.transform(X_train[X_num])
X_test  = numeric_transformer.transform(X_test[X_num])

print(type(X_train))
print(type(X_test))

In [0]:
type(X_train)

## Deep Neural Networks

**Ejercicios**
1. Construya, por medio de `tensorflow`, una regresión logística que trabaje con la métrica [AUC (Área bajo la curva ROC)](https://keras.io/api/metrics/classification_metrics/#auc-class).
2. Construya, por medio de `tensorflow`, una red neuronal profunda que trabaje con la [AUC (Área bajo la curva ROC)](https://keras.io/api/metrics/classification_metrics/#auc-class).
3. Construya una red neuronal profunda con al menos 5 capas ocultas y gran número de neuronas en cada capa. Para evitar overfitting, utilice [regularización](https://faroit.com/keras-docs/1.2.2/regularizers/) en las capas ocultas (``l1`` o ``l2``) y Dropout. Juegue con distintas tasas de regularización y comente los efectos sobre la [métrica AUC ](https://keras.io/api/metrics/classification_metrics/#auc-class).
4. Para el modelo del punto 3, cálcule la métrica AUC a partir del Test Set. ¿Qué podemos decir acerca del grado de overfitting?

**Nota:**


Queda enteramente a su criterio el definir los hiperparámetros del modelo. Utilice el enfoque del **set de validación** para garantizar que los modelos no tengan **overfitting**.


**Conclusión:**


En base a los resultados obtenidos, ¿cuál de los algoritmos propuestos para el proyecto final tiene mejor performance?

In [0]:
#Su código aquí
